La librería python-jobspy agrega resultados de LinkedIn, Indeed, Glassdoor y ZipRecruiter sin que tengas que configurar Selenium.

In [7]:
#Librerías necesarias
import pandas as pd
from jobspy import scrape_jobs
import time
import random

In [8]:
# 1. Configuración de la fragmentación
locations = ["Málaga, Spain", "Granada, Spain"]
jobs_presencial = []

print("Iniciando búsqueda fragmentada...")

for loc in locations:
    print(f"--- Extrayendo ofertas en: {loc} ---")
    
    try:
        jobs = scrape_jobs(
            site_name=["linkedin"],
            search_term="Data Engineer", # Cambia el puesto si es necesario
            location=loc,
            results_wanted=50,           # Cantidad moderada por ciudad
            is_remote=False,             # Prioriza presencial/híbrido
            linkedin_fetch_description=True 
        )
        
        # Convertir a DataFrame y añadir columna de origen para control
        df_loc = pd.DataFrame(jobs)
        if not df_loc.empty:
            df_loc['search_location'] = loc
            jobs_presencial.append(df_loc)
            print(f"Se han encontrado {len(df_loc)} ofertas en {loc}.")
        else:
            print(f"No se encontraron ofertas recientes en {loc}.")

    except Exception as e:
        print(f"Error al buscar en {loc}: {e}")

    wait_time = random.uniform(10, 20)
    print(f"Esperando {wait_time:.2f} segundos antes de la siguiente ciudad...\n")
    time.sleep(wait_time)


# 3. Consolidación de datos
if jobs_presencial:
    df_presencial = pd.concat(jobs_presencial, ignore_index=True)
    
    # Filtrar explícitamente por modalidad si el dataset trae la columna
    # (A veces LinkedIn etiqueta 'Hybrid' o 'On-site' en la descripción o metadatos)
    if 'job_type' in df_presencial.columns:
        # Aquí podrías filtrar si quisieras ser aún más estricto
        pass

    # Eliminar duplicados por URL (por si una oferta aparece en ambas ciudades)
    df_presencial = df_presencial.drop_duplicates(subset=['job_url'])
else:
    print("No se pudo recolectar ninguna oferta.")

Iniciando búsqueda fragmentada...
--- Extrayendo ofertas en: Málaga, Spain ---


2026-02-15 16:31:40,533 - INFO - JobSpy:Linkedin - finished scraping


Se han encontrado 50 ofertas en Málaga, Spain.
Esperando 13.12 segundos antes de la siguiente ciudad...

--- Extrayendo ofertas en: Granada, Spain ---
Se han encontrado 40 ofertas en Granada, Spain.
Esperando 13.90 segundos antes de la siguiente ciudad...



In [9]:
# 1. Configuración para la búsqueda en remoto
print("--- Iniciando búsqueda de ofertas en REMOTO ---")

all_jobs = [df_presencial] if not df_presencial.empty else []

try:
    jobs_remote = scrape_jobs(
        site_name=["linkedin"],
        search_term="Data Engineer", # Mantén el mismo término para coherencia
        location="Spain",            # En remoto buscamos en todo el país
        results_wanted=50, 
        is_remote=True,              # FILTRO CLAVE: Solo remoto
        linkedin_fetch_description=True 
    )
    
    df_remote = pd.DataFrame(jobs_remote)
    
    if not df_remote.empty:
        df_remote['search_location'] = 'Remote (Spain)'
        all_jobs.append(df_remote)
        print(f"Se han encontrado {len(df_remote)} ofertas en remoto.")
    else:
        print("No se encontraron ofertas en remoto.")

except Exception as e:
    print(f"Error en la búsqueda remota: {e}")

# 2. Consolidación final (Igual que antes pero ahora incluye lo remoto)
if all_jobs:
    df_final = pd.concat(all_jobs, ignore_index=True)
    
    # Limpieza crucial: Eliminar duplicados
    # A veces una oferta de Málaga también se etiqueta como Remoto Nacional
    antes = len(df_final)
    df_final = df_final.drop_duplicates(subset=['job_url'])
    despues = len(df_final)
    
    print(f"\nProceso completado.")
    print(f"Total bruto: {antes} | Total tras limpiar duplicados: {despues}")

--- Iniciando búsqueda de ofertas en REMOTO ---
Se han encontrado 50 ofertas en remoto.

Proceso completado.
Total bruto: 140 | Total tras limpiar duplicados: 140


In [10]:
import os
from datetime import datetime

# 1. Obtener la fecha para la carpeta (dd-mm-yyyy)
fecha_hoy = datetime.now().strftime("%d-%m-%Y")

# 2. Obtener el timestamp para el nombre del archivo (HH-MM)
timestamp_archivo = datetime.now().strftime("%H-%M")

# 3. Definir la ruta completa: scraps/dd-mm-yyyy/
# Usamos os.path.join para que funcione bien en cualquier sistema operativo
ruta_carpeta = os.path.join('scraps', fecha_hoy)

# 4. Crear la carpeta (y las carpetas padre si no existen)
if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)

# 5. Definir el nombre del archivo y guardar
nombre_archivo = f"ofertas_it_{timestamp_archivo}.csv"
ruta_final = os.path.join(ruta_carpeta, nombre_archivo)

if not df_final.empty:
    df_final.to_csv(ruta_final, index=False, encoding='utf-8')
    print(f"--- Datos guardados en: {ruta_final} ---")
else:
    print("El DataFrame está vacío, no se guardó el archivo.")

--- Datos guardados en: scraps/15-02-2026/ofertas_it_16-34.csv ---
